# CNN Classifier

In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.metrics import f1_score
import time

# Hyperparameters
learning_rate = 0.001
batch_size = 64
num_epochs = 10

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize images to [-1, 1]
])

# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Define CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize the model, criterion, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the CNN Model and Logging Metrics
train_start_time = time.time()
train_losses = []
f1_scores_train = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    true_labels = []
    predicted_labels = []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calculate F1 score
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())

    avg_loss = total_loss / len(train_loader)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')

    train_losses.append(avg_loss)
    f1_scores_train.append(f1)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, F1 Score: {f1:.4f}")

train_end_time = time.time()
train_time_cnn = train_end_time - train_start_time

# Evaluate on Test Data
model.eval()
true_labels = []
predicted_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())

test_accuracy_cnn = 100 * sum(np.array(true_labels) == np.array(predicted_labels)) / len(true_labels)
f1_score_cnn = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Test Accuracy (CNN): {test_accuracy_cnn:.2f}%")
print(f"Test F1 Score (CNN): {f1_score_cnn:.4f}")
print(f"Training Time (CNN): {train_time_cnn:.2f} seconds")

Epoch [1/10], Loss: 0.2104, F1 Score: 0.9370
Epoch [2/10], Loss: 0.0829, F1 Score: 0.9749
Epoch [3/10], Loss: 0.0624, F1 Score: 0.9814
Epoch [4/10], Loss: 0.0470, F1 Score: 0.9856
Epoch [5/10], Loss: 0.0398, F1 Score: 0.9877
Epoch [6/10], Loss: 0.0347, F1 Score: 0.9890
Epoch [7/10], Loss: 0.0293, F1 Score: 0.9904
Epoch [8/10], Loss: 0.0242, F1 Score: 0.9920
Epoch [9/10], Loss: 0.0237, F1 Score: 0.9921
Epoch [10/10], Loss: 0.0198, F1 Score: 0.9932
Test Accuracy (CNN): 99.24%
Test F1 Score (CNN): 0.9924
Training Time (CNN): 141.10 seconds


# AlexNet Model

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

# Load AlexNet pretrained model
model_alexnet = models.alexnet(pretrained=True)

# Modify the first convolutional layer to accept 1 input channel (grayscale)
model_alexnet.features[0] = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)

# Modify the classifier part for 10 output classes (MNIST has 10 classes)
model_alexnet.classifier[6] = nn.Linear(4096, 10)

# Define the loss function (CrossEntropyLoss) and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_alexnet.parameters(), lr=0.001)

# Move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_alexnet.to(device)

# Define the transformations for MNIST dataset (resize to 224x224)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Training loop
for epoch in range(5):
    model_alexnet.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model_alexnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/5], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/5], Loss: 2.3054
Epoch [2/5], Loss: 2.3022
Epoch [3/5], Loss: 2.3018
Epoch [4/5], Loss: 2.3017
Epoch [5/5], Loss: 2.3016


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Evaluation on Test Data
model_alexnet.eval()  # Set model to evaluation mode
true_labels = []
predicted_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model_alexnet(images)

        # Get predictions
        _, preds = torch.max(outputs, 1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())

# Calculate accuracy
accuracy = 100 * sum(np.array(true_labels) == np.array(predicted_labels)) / len(true_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Test Accuracy (AlexNet): {accuracy:.2f}%")
print(f"Test F1 Score (AlexNet): {f1:.4f}")

Test Accuracy (AlexNet): 11.35%
Test F1 Score (AlexNet): 0.0231


# VGG16 Model

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

# Load the pretrained VGG16 model
model_vgg16 = models.vgg16(pretrained=True)

# Modify the first convolutional layer to accept 1 input channel (grayscale)
model_vgg16.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)

# Replace the classifier part to suit our dataset (MNIST - 10 classes)
model_vgg16.classifier[6] = nn.Linear(4096, 10)

# Move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_vgg16.to(device)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_vgg16.parameters(), lr=0.001)

# Define the transformations for MNIST dataset (resize to 224x224)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Training loop
for epoch in range(5):
    model_vgg16.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model_vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/5], Loss: {running_loss/len(train_loader):.4f}")


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 233MB/s] 


Epoch [1/5], Loss: 0.2734
Epoch [2/5], Loss: 0.1817
Epoch [3/5], Loss: 1.2427
Epoch [4/5], Loss: 2.3018
Epoch [5/5], Loss: 2.3014


In [ ]:
# Evaluation on Test Data
model_vgg16.eval()  # Set model to evaluation mode
true_labels = []
predicted_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model_vgg16(images)

        # Get predictions
        _, preds = torch.max(outputs, 1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())

# Calculate accuracy
accuracy = 100 * sum(np.array(true_labels) == np.array(predicted_labels)) / len(true_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Test Accuracy (VGG16): {accuracy:.2f}%")
print(f"Test F1 Score (VGG16): {f1:.4f}")

Test Accuracy (VGG16): 11.35%
Test F1 Score (VGG16): 0.0231


# Vision Transformer (VIT)

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from tqdm import tqdm, trange

In [ ]:
# Hyperparameters
PATCH_SIZE = 7
N_BLOCKS = 2
HIDDEN_DIM = 64
N_HEADS = 4
EPOCHS = 5
LR = 0.001

In [ ]:
# Patchify function
def patchify(images, n_patches):
    n, c, h, w = images.shape
    assert h == w, "Patchify method is implemented for square images only"
    patches = torch.zeros(n, n_patches**2, h * w // n_patches**2)
    patch_size = h // n_patches
    for idx, image in enumerate(images):
        for i in range(n_patches):
            for j in range(n_patches):
                patch = image[
                    :,
                    i * patch_size : (i + 1) * patch_size,
                    j * patch_size : (j + 1) * patch_size,
                ]
                patches[idx, i * n_patches + j] = patch.flatten()
    return patches

In [ ]:
# Multi-head self-attention layer
class MyMSA(nn.Module):
    def __init__(self, d, n_heads):
        super(MyMSA, self).__init__()
        self.d = d
        self.n_heads = n_heads
        d_head = int(d / n_heads)
        self.q_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.k_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.v_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.d_head = d_head
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, sequences):
        result = []
        for sequence in sequences:
            seq_result = []
            for head in range(self.n_heads):
                q_mapping = self.q_mappings[head]
                k_mapping = self.k_mappings[head]
                v_mapping = self.v_mappings[head]

                seq = sequence[:, head * self.d_head : (head + 1) * self.d_head]
                q, k, v = q_mapping(seq), k_mapping(seq), v_mapping(seq)

                attention = self.softmax(q @ k.T / (self.d_head ** 0.5))
                seq_result.append(attention @ v)
            result.append(torch.hstack(seq_result))
        return torch.cat([torch.unsqueeze(r, dim=0) for r in result])

In [ ]:
# ViT Block
class MyViTBlock(nn.Module):
    def __init__(self, hidden_d, n_heads, mlp_ratio=4):
        super(MyViTBlock, self).__init__()
        self.hidden_d = hidden_d
        self.n_heads = n_heads
        self.norm1 = nn.LayerNorm(hidden_d)
        self.mhsa = MyMSA(hidden_d, n_heads)
        self.norm2 = nn.LayerNorm(hidden_d)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_d, mlp_ratio * hidden_d),
            nn.GELU(),
            nn.Linear(mlp_ratio * hidden_d, hidden_d),
        )

    def forward(self, x):
        out = x + self.mhsa(self.norm1(x))
        out = out + self.mlp(self.norm2(out))
        return out


In [ ]:
# Vision Transformer Model
class MyViT(nn.Module):
    def __init__(self, chw, n_patches=7, n_blocks=2, hidden_d=8, n_heads=2, out_d=10):
        super(MyViT, self).__init__()
        self.chw = chw
        self.n_patches = n_patches
        self.n_blocks = n_blocks
        self.n_heads = n_heads
        self.hidden_d = hidden_d

        self.patch_size = (chw[1] // n_patches, chw[2] // n_patches)
        self.input_d = int(chw[0] * self.patch_size[0] * self.patch_size[1])

        self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)
        self.class_token = nn.Parameter(torch.rand(1, self.hidden_d))
        self.positional_embeddings = nn.Parameter(torch.randn(n_patches ** 2 + 1, hidden_d))

        self.blocks = nn.ModuleList([MyViTBlock(hidden_d, n_heads) for _ in range(n_blocks)])

        self.mlp = nn.Sequential(nn.Linear(self.hidden_d, out_d), nn.Softmax(dim=-1))

    def forward(self, images):
        n, c, h, w = images.shape
        patches = patchify(images, self.n_patches).to(self.positional_embeddings.device)
        tokens = self.linear_mapper(patches)

        tokens = torch.cat((self.class_token.expand(n, 1, -1), tokens), dim=1)
        out = tokens + self.positional_embeddings

        for block in self.blocks:
            out = block(out)

        out = out[:, 0]
        return self.mlp(out)

In [ ]:
def main():
    # Loading data
    transform = ToTensor()

    train_set = MNIST(
        root="./../datasets", train=True, download=True, transform=transform
    )
    test_set = MNIST(
        root="./../datasets", train=False, download=True, transform=transform
    )

    train_loader = DataLoader(train_set, shuffle=True, batch_size=128)
    test_loader = DataLoader(test_set, shuffle=False, batch_size=128)

    # Defining model and training options
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(
        "Using device: ",
        device,
        f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "",
    )
    model = MyViT(
        (1, 28, 28), n_patches=7, n_blocks=2, hidden_d=8, n_heads=2, out_d=10
    ).to(device)
    N_EPOCHS = 5
    LR = 0.005

    # Training loop
    optimizer = Adam(model.parameters(), lr=LR)
    criterion = CrossEntropyLoss()
    for epoch in trange(N_EPOCHS, desc="Training"):
        train_loss = 0.0
        for batch in tqdm(
            train_loader, desc=f"Epoch {epoch + 1} in training", leave=False
        ):
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)

            train_loss += loss.detach().cpu().item() / len(train_loader)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{N_EPOCHS} loss: {train_loss:.2f}")

    # Test loop
    with torch.no_grad():
        correct, total = 0, 0
        test_loss = 0.0
        for batch in tqdm(test_loader, desc="Testing"):
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)
            test_loss += loss.detach().cpu().item() / len(test_loader)

            correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
            total += len(x)
        print(f"Test loss: {test_loss:.2f}")
        print(f"Test accuracy: {correct / total * 100:.2f}%")

In [ ]:
if __name__ == "__main__":
    main()

Using device:  cuda (Tesla P100-PCIE-16GB)


Training:  20%|██        | 1/5 [05:02<20:09, 302.37s/it]              

Epoch 1/5 loss: 2.08



Training:  40%|████      | 2/5 [10:04<15:06, 302.29s/it]              

Epoch 2/5 loss: 1.83


...

Epoch 4/5 loss: 1.76



Training: 100%|██████████| 5/5 [25:13<00:00, 302.78s/it]              


Epoch 5/5 loss: 1.74


Testing: 100%|██████████| 79/79 [00:30<00:00,  2.60it/s]

Test loss: 1.70
Test accuracy: 76.07%
